# 1.2 By Randomized Measurement
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_randomized = EntropyMeasure()
# It's default method. EntropyMeasure(method='randomized') also works

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_randomized.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_randomized.add(GHZ(i), f"ghz_{i}")
    experiment_randomized.add(make_neel_circuit(i), f"neel_{i}")

experiment_randomized.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x779018ba7530>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x779018cba270>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x779019165a30>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x779074addc10>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x779018b27da0>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x77901996e1b0>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x779018a2f920>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x779018a2fe30>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x779018a60050>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x779018a60110>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x779018a60170>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.randomized_measure.arguments import (
    EntropyMeasureRandomizedMeasureArgs,
)

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureRandomizedMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function :func:`generate_random_unitary_seeds` 
    in :mod:`qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds

        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[EntropyMeasureRandomizedMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "times": 100,
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_randomized.multiOutput(
    config_list,
    summoner_name="qurrent.randomized_measure",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| qurrent.randomized_measure.001 is repeat location.
| qurrent.randomized_measure.002 is repeat location.
| qurrent.randomized_measure.003 is repeat location.
| qurrent.randomized_measure.004 is repeat location.
| qurrent.randomized_measure.005 is repeat location.
| qurrent.randomized_measure.006 is repeat location.
| qurrent.randomized_measure.007 is repeat location.
| qurrent.randomized_measure.008 is repeat location.
| qurrent.randomized_measure.009 is repeat location.
| qurrent.randomized_measure.010 is repeat location.
| Write "qurrent.randomized_measure.011", at location "qurrent.randomized_measure.011"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for 54194c63-729e-4c49-b348-6667f59f2dac


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.011/qurryinfo.json...
| Exporting qurrent.randomized_measure.011/qurryinfo.json done.


'54194c63-729e-4c49-b348-6667f59f2dac'

You can check the result of `multiOutput` that we just executed by accessing the `.multimanagers`

In [6]:
experiment_randomized.multimanagers

MultiManagerContainer(num=1, {
  "54194c63-729e-4c49-b348-6667f59f2dac":
    <MultiManager(name="qurrent.randomized_measure.011", jobstype="local", ..., exps_num=180)>,
})

In [7]:
experiment_randomized.multimanagers[multi_exps1]

<MultiManager(id="54194c63-729e-4c49-b348-6667f59f2dac",
  name="qurrent.randomized_measure.011",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-06-13 21:33:21',},
  exps_num=180)>

### d. Run post-processing at once


In [8]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 0/180 - Analysis:  - 00:00 < ?

| "report.001" has been completed.


'54194c63-729e-4c49-b348-6667f59f2dac'

In [9]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"]

TagList(list,
        {('trivial_paramagnet',
          'size_2'): [{'purity': np.float64(0.9647203254699707),
           'entropy': np.float64(0.0518173323382352),
           'puritySD': np.float64(0.5902960074566418),
           'entropySD': np.float64(0.8827606303405913),
           'num_classical_registers': 2,
           'classical_registers': [0, 1],
           'classical_registers_actually': [0, 1],
           'all_system_source': 'independent',
           'purityAllSys': np.float64(0.9647203254699707),
           'entropyAllSys': np.float64(0.0518173323382352),
           'puritySDAllSys': np.float64(0.5902960074566418),
           'entropySDAllSys': np.float64(0.8827606303405913),
           'num_classical_registers_all_sys': 2,
           'classical_registers_all_sys': None,
           'classical_registers_actually_all_sys': [0, 1],
           'errorRate': np.float64(0.023803076239245307),
           'mitigatedPurity': np.float64(1.0000000000000002),
           'mitigatedEntr

In [10]:
print("| Available results:")
for k, v in (
    experiment_randomized.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


In [11]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][0]

{'purity': np.float64(0.9728394508361816),
 'entropy': np.float64(0.039726360378400896),
 'puritySD': np.float64(0.6007339677596828),
 'entropySD': np.float64(0.8908725025856153),
 'num_classical_registers': 10,
 'classical_registers': [0, 1],
 'classical_registers_actually': [0, 1],
 'all_system_source': 'independent',
 'purityAllSys': np.float64(1.72611629486084),
 'entropyAllSys': np.float64(-0.7875296675104797),
 'puritySDAllSys': np.float64(1.3215819929906603),
 'entropySDAllSys': np.float64(1.1045836210992332),
 'num_classical_registers_all_sys': 10,
 'classical_registers_all_sys': None,
 'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'errorRate': np.float64(-0.31408754881872725),
 'mitigatedPurity': np.float64(0.6685942387094294),
 'mitigatedEntropy': np.float64(0.5807971716118946),
 'counts_num': 100,
 'taking_time': 0.000533008,
 'taking_time_all_sys': 0.024657983,
 'counts_used': None,
 'input': {'num_qubits': 10,
  'selected_qubits': [0, 1],
  'regi

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [12]:
expkeys_of_multi_exps1 = list(
    experiment_randomized.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [13]:
expkeys_of_multi_exps1

['44ef3f36-04df-4c69-8b7d-5088bbdaeaf8',
 '18434286-cb0c-40fa-9110-17979cedf576',
 '37b5baed-cd88-4184-9be6-fbd771ec987f',
 '9a3889e7-662d-4535-8cb8-b71e1f305916',
 '5be58787-c8af-479f-9c90-5ac824988f63',
 'e4c88489-ab51-4c93-8b08-2b086a339ef6',
 '7de970e8-8976-4f58-a4f9-9324b39dac5f',
 '28ebdea7-cd75-4815-93f6-67e48f82c4db',
 'b493860d-b318-4ebf-8ef5-c29c44376e2c',
 'd8788c64-abeb-4ddd-bf3f-ffb083aabebf',
 '155ab54b-d291-40d9-9e03-60788521e433',
 'add65670-9781-4b11-8423-eda705b7ab76',
 'e5609560-dfd8-4a54-b884-0102aa46c1ed',
 '3052b447-f70d-497c-95c8-81878de5e5db',
 'bb24d88c-69de-49bd-abd8-7c850c17e560',
 'bc5c8842-dfb4-4066-a46d-c8f29e9d29ca',
 '32c6384f-f8b4-4a32-af0f-247e81a8922f',
 '549d7116-3567-4548-8a43-554f22b14f17',
 '2e1c9a98-beab-4095-a8a8-4b795287ef25',
 '850be13e-ca36-4371-819f-f00f97beca0c',
 '038a83bf-818f-40ce-8ed9-87cf426b401d',
 '4050b15c-be0c-4bef-bb49-2c94692d7771',
 'a7ae8939-a8c7-441a-ac0d-90dcaa6ef1c4',
 'cd65cfad-655d-4544-9873-560170317fa4',
 'a7ec8846-d937-

In [14]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={
        k: (
            {
                "selected_qubits": [0, 1],
            }
            if idx < 3
            else False
        )
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 0/180 - Analysis:  - 00:00 < ?

| "first_3.002" has been completed.


'54194c63-729e-4c49-b348-6667f59f2dac'

In [15]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [16]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={
        k: {
            "selected_qubits": [0, 1],  # selected qubits for the analysis
        }
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 0/180 - Analysis:  - 00:00 < ?

| "all_manual.003" has been completed.
| Export multimanager...


| 0/9 - Exporting MultiManager content... - 00:00 < ?

| 0/3 - exporting quantity - 00:00 < ?

| Export multi.config.json for 54194c63-729e-4c49-b348-6667f59f2dac


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.011/qurryinfo.json...
| Exporting qurrent.randomized_measure.011/qurryinfo.json done.


'54194c63-729e-4c49-b348-6667f59f2dac'

In [17]:
experiment_randomized.multimanagers[multi_exps1].quantity_container.keys()

dict_keys(['report.001', 'first_3.002', 'all_manual.003'])

In [18]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [19]:
multi_exps1_reades = experiment_randomized.multiRead(
    save_location=".",
    summoner_name="qurrent.randomized_measure.001",
)

| Retrieve qurrent.randomized_measure.001...
| at: qurrent.randomized_measure.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

---

### Post-Process Availablities and Version Info

In [20]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No    
 -